# Create scheduler pickles for testing `schedview`

In [1]:
from collections import namedtuple
import lzma
import pickle
from pathlib import Path
import warnings

In [2]:
import numpy as np
import pandas as pd
import healpy as hp
import rubin_sim
from astropy.time import Time
import astropy.units as u
import astropy.coordinates
from rubin_sim.scheduler.model_observatory import ModelObservatory
from rubin_sim.scheduler.surveys.field_survey import FieldSurvey
from rubin_sim.scheduler.schedulers import CoreScheduler
import rubin_sim.scheduler.basis_functions as bf
from rubin_sim.scheduler import sim_runner
import schedview.collect
import schedview.compute
from rubin_sim.utils import survey_start_mjd

In [3]:
SchedulerPickleContent = namedtuple('SchedulerPickleContent', ['scheduler', 'conditions'])

In [4]:
mjd_start = survey_start_mjd()
model_observatory = ModelObservatory(mjd_start=mjd_start)
nside = model_observatory.nside
survey_length = (4*u.hour).to(u.day).value

# Create a near-baseline sample scheduler

Use the `make_sample_test_data.py` script in `schedview`, found in `util/sample_data`:

# Get a production auxtel scheduler

Use python `efd_sched.py` script in `schedview`, found in `util`

# Create schedulers with problems

## Introduction

I start by making an example scheduler with two tiers. The first tier is an equatorial survey in with fields every hour, such that there should be fields visible at all times.
The second tier is a single greed survey (in g) covering the whole sky.

## Make some basis functions

In [5]:
def make_sky_bf_list(band='g', nside=32):
    not_twilight = bf.feasibility_funcs.NotTwilightBasisFunction()
    moon_limit = bf.mask_basis_funcs.MoonAvoidanceBasisFunction(nside=nside, moon_distance=30.0)
    zenith_limit = bf.mask_basis_funcs.ZenithShadowMaskBasisFunction(nside=nside, min_alt=20.0, max_alt=82.0)
    sky_brightness_limit = bf.basis_functions.SkybrightnessLimitBasisFunction(nside=nside, filtername=band, sbmin=18.5, sbmax=30)
    wind_limit = bf.basis_functions.AvoidDirectWind(5)
    m5diff = bf.basis_functions.M5DiffBasisFunction(filtername=band, nside=nside)
    basis_functions = [
        not_twilight,
        moon_limit,
        zenith_limit,
        sky_brightness_limit,
        wind_limit,
        m5diff
    ]
    return basis_functions

In [6]:
def make_field_bf_list(ra, decl, band='g', nside=32):
    basis_functions = make_sky_bf_list(band=band, nside=nside)
    basis_functions.append(bf.feasibility_funcs.HourAngleLimitBasisFunction(RA=ra, ha_limits=[[22,24], [0,2]]))
    return basis_functions

In [7]:
def make_field_survey(ra, decl, band='g', nside=32):
    basis_functions = make_field_bf_list(ra, decl, band=band, nside=nside)
    sequence = band
    nvis = [1]*len(band)
    survey_name = f"field_{ra}_{'n' if decl<0 else 'p'}{np.abs(decl)}_{band}"
    survey = FieldSurvey(basis_functions, np.array([ra]), np.array([decl]), sequence=sequence, nvis=nvis, nside=nside, survey_name=survey_name, reward_value=1.0)
    return survey

## Make an equatorial survey in g

Start by making a list of field surveys in g on the equator, with fields spaced every hour (15 degrees):

In [8]:
decl = 0
band = 'g'
field_surveys = [make_field_survey(ra, decl, band, nside) for ra in range(0, 360, 15)]

As a fallback, greate a greedy survey in g that covers the whole sky:

In [9]:
sky_basis_functions = make_sky_bf_list(band=band, nside=nside)
weights = [1] * len(sky_basis_functions)
greedy_surveys = [rubin_sim.scheduler.surveys.surveys.GreedySurvey(sky_basis_functions, weights, filtername=band, survey_name=f"greedy_{band}")]

Actually create the scheduler:

In [10]:
scheduler = CoreScheduler([field_surveys, greedy_surveys], nside=nside)

Configure the scheduler to keep reward values:

In [11]:
scheduler.keep_rewards = True

Actually run the survey for a little bit:

In [12]:
observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
    model_observatory,
    scheduler,
    mjd_start=mjd_start,
    survey_length=survey_length,
    record_rewards=True,
)
conditions = scheduler.conditions

progress = 75.46%Skipped 0 observations
Flushed 0 observations from queue for being stale
Completed 397 observations
ran in 0 min = 0.0 hours


Save the survey:

In [13]:
output_tuple = (scheduler, conditions)
fname = Path('../tmp/eq_field_survey_v0.p.xz').resolve()
with lzma.open(fname, 'wb') as sched_out:
    pickle.dump(output_tuple, sched_out)

fname

PosixPath('/sdf/data/rubin/user/neilsen/devel/schedview/tmp/eq_field_survey_v0.p.xz')

In [14]:
print(f"Conditions calculated for {Time(conditions.mjd[0], format='mjd').iso} (mjd {conditions.mjd[0]})")

Conditions calculated for 2025-05-01 03:59:35.041 (mjd 60796.16637779371)


Take a look at one survey:

In [15]:
scheduler_reward_df = scheduler.make_reward_df(conditions, accum=True)
scheduler_reward_df.loc[(0,1),:]

basis_function  \
list_index survey_index                         
0          1                      NotTwilight   
           1                    MoonAvoidance   
           1                 ZenithShadowMask   
           1             SkybrightnessLimit g   
           1                  AvoidDirectWind   
           1                         M5Diff g   
           1                   HourAngleLimit   

                                    basis_function_class  feasible  \
list_index survey_index                                              
0          1                    NotTwilightBasisFunction      True   
           1                  MoonAvoidanceBasisFunction      True   
           1               ZenithShadowMaskBasisFunction     False   
           1             SkybrightnessLimitBasisFunction     False   
           1                             AvoidDirectWind      True   
           1                         M5DiffBasisFunction     False   
           1                 HourAngleLimitBasisFunction     False   

                         max_basis_reward  basis_area  basis_weight  \
list_index survey_index                                               
0          1                          0.0    3.357175      0.142857   
           1                          1.0    3.357175      0.142857   
           1                         -inf    0.000000      0.142857   
           1                         -inf    0.000000      0.142857   
           1                          0.0    3.357175      0.142857   
           1                         -inf    0.000000      0.142857   
           1                         -inf    3.357175      0.142857   

                         max_accum_reward    accum_area tier_label  \
list_index survey_index                                              
0          1                     0.000000  41252.961249     tier 0   
           1                     0.142857  41252.961249     tier 0   
           1                         -inf      0.000000     tier 0   
           1                         -inf      0.000000     tier 0   
           1                         -inf      0.000000     tier 0   
           1                         -inf      0.000000     tier 0   
           1                         -inf      0.000000     tier 0   

                          survey_label survey_class  survey_reward  
list_index survey_index                                             
0          1             field_15_p0_g  FieldSurvey           -inf  
           1             field_15_p0_g  FieldSurvey           -inf  
           1             field_15_p0_g  FieldSurvey           -inf  
           1             field_15_p0_g  FieldSurvey           -inf  
           1             field_15_p0_g  FieldSurvey           -inf  
           1             field_15_p0_g  FieldSurvey           -inf  
           1             field_15_p0_g  FieldSurvey           -inf

Take a look at the rewards for all surveys.

In [16]:
scheduler_summary_df = schedview.compute.make_scheduler_summary_df(scheduler, conditions, scheduler_reward_df)
scheduler_summary_df.query('tier=="tier 0"')

,tier,survey_name,survey_url,reward
0,tier 0,0: field_0_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"ZenithShadowMask, SkybrightnessLimit g, M5Diff..."
1,tier 0,10: field_150_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,HourAngleLimit
2,tier 0,11: field_165_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,HourAngleLimit
3,tier 0,12: field_180_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,0.467255
4,tier 0,13: field_195_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,0.480597
5,tier 0,14: field_210_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,0.482711
6,tier 0,15: field_225_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,0.474278
7,tier 0,16: field_240_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,HourAngleLimit
8,tier 0,17: field_255_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,HourAngleLimit
9,tier 0,18: field_270_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"SkybrightnessLimit g, M5Diff g, HourAngleLimit"


# Make a survey where everything is infeasible

Create a combination of survey and conditions where everything is infeasible, blocked either by wind or by sky brightess, but where there are observations not blocked by both.

The first full moon after the nominal survey start is 2025-05-12. This should be in the default limited sky brightness data set.

Find the next full moon after the start of the survey:

In [17]:
month_mjds = np.arange(int(mjd_start), int(mjd_start)+30)
month_phases = np.array([model_observatory.almanac.get_sun_moon_positions(mjd)['moon_phase'] for mjd in month_mjds])
mjd = month_mjds[np.argmax(month_phases)]
Time(mjd, format='mjd').iso

'2025-05-13 00:00:00.000'

Find a time during the night where the moon is at a zd of about 55 degrees.

In [18]:
sunset_info = model_observatory.almanac.get_sunset_info(mjd)
sunset, sunrise = sunset_info[3], sunset_info[4]
for mjd in np.arange(sunset, sunrise, 1.0/(24*4)):
    moon_alt = model_observatory.almanac.get_sun_moon_positions(mjd)['moon_alt']
    if np.abs(np.degrees(moon_alt)-55) < 5:
        break

time_to_sched = Time(mjd, format='mjd')
time_to_sched.iso, mjd

('2025-05-13 02:06:31.806', 60808.08786812174)

In [19]:
model_observatory.mjd = time_to_sched.mjd

Set the wind so it comes from the opposite direction.

In [20]:
moon_az = model_observatory.almanac.get_sun_moon_positions(time_to_sched.mjd)['moon_az']
wind_az = moon_az - np.pi ;# opposite moon
wind_data = rubin_sim.site_models.ConstantWindData(wind_speed=18.0, wind_direction=wind_az)
model_observatory.wind_data = wind_data

Actually build our instance of Conditions:

In [21]:
conditions = model_observatory.return_conditions()

In [22]:
scheduler.make_reward_df(conditions, accum=True)

/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarnin

basis_function  \
list_index survey_index                         
0          0                      NotTwilight   
           0                    MoonAvoidance   
           0                 ZenithShadowMask   
           0             SkybrightnessLimit g   
           0                  AvoidDirectWind   
...                                       ...   
1          0                    MoonAvoidance   
           0                 ZenithShadowMask   
           0             SkybrightnessLimit g   
           0                  AvoidDirectWind   
           0                         M5Diff g   

                                    basis_function_class  feasible  \
list_index survey_index                                              
0          0                    NotTwilightBasisFunction      True   
           0                  MoonAvoidanceBasisFunction      True   
           0               ZenithShadowMaskBasisFunction     False   
           0             SkybrightnessLimitBasisFunction     False   
           0                             AvoidDirectWind      True   
...                                                  ...       ...   
1          0                  MoonAvoidanceBasisFunction      True   
           0               ZenithShadowMaskBasisFunction      True   
           0             SkybrightnessLimitBasisFunction      True   
           0                             AvoidDirectWind      True   
           0                         M5DiffBasisFunction      True   

                         max_basis_reward    basis_area  basis_weight  \
list_index survey_index                                                 
0          0                     0.000000      3.357175      0.142857   
           0                     1.000000      3.357175      0.142857   
           0                         -inf      0.000000      0.142857   
           0                         -inf      0.000000      0.142857   
           0                   -52.644817      3.357175      0.142857   
...                                   ...           ...           ...   
1          0                     1.000000  38483.292220      1.000000   
           0                     1.000000  13334.697435      1.000000   
           0                     1.000000   2141.877383      1.000000   
           0                    -0.000201  24329.444187      1.000000   
           0                    -1.634773  12045.542396      1.000000   

                         max_accum_reward    accum_area tier_label  \
list_index survey_index                                              
0          0                     0.000000  41252.961249     tier 0   
           0                     0.142857  41252.961249     tier 0   
           0                         -inf      0.000000     tier 0   
           0                         -inf      0.000000     tier 0   
           0                         -inf      0.000000     tier 0   
...                                   ...           ...        ...   
1          0                     1.000000  38483.292220     tier 1   
           0                     2.000000  10601.957326     tier 1   
           0                     3.000000   2141.877383     tier 1   
           0                         -inf      0.000000     tier 1   
           0                         -inf      0.000000     tier 1   

                         survey_label  survey_class  survey_reward  
list_index survey_index                                             
0          0             field_0_p0_g   FieldSurvey           -inf  
           0             field_0_p0_g   FieldSurvey           -inf  
           0             field_0_p0_g   FieldSurvey           -inf  
           0             field_0_p0_g   FieldSurvey           -inf  
           0             field_0_p0_g   FieldSurvey           -inf  
...                               ...           ...            ...  
1          0                 greedy_g  GreedySurvey            NaN  


In [23]:
schedview.compute.make_scheduler_summary_df(scheduler, conditions)

/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarning: All-NaN axis encountered
  survey_df["survey_reward"] = np.nanmax(survey.calc_reward_function(conditions))
/sdf/data/rubin/user/neilsen/devel/rubin_sim/rubin_sim/scheduler/schedulers/core_scheduler.py:547: RuntimeWarnin

,tier,survey_name,survey_url,reward
0,tier 0,0: field_0_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"ZenithShadowMask, SkybrightnessLimit g, M5Diff..."
1,tier 0,10: field_150_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"AvoidDirectWind, HourAngleLimit"
2,tier 0,11: field_165_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,AvoidDirectWind
3,tier 0,12: field_180_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"SkybrightnessLimit g, AvoidDirectWind"
4,tier 0,13: field_195_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,SkybrightnessLimit g
5,tier 0,14: field_210_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,SkybrightnessLimit g
6,tier 0,15: field_225_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"MoonAvoidance, SkybrightnessLimit g, HourAngle..."
7,tier 0,16: field_240_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"MoonAvoidance, SkybrightnessLimit g, HourAngle..."
8,tier 0,17: field_255_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"SkybrightnessLimit g, M5Diff g, HourAngleLimit"
9,tier 0,18: field_270_p0_g,https://rubin-sim.lsst.io/api/FieldSurvey.html...,"ZenithShadowMask, SkybrightnessLimit g, M5Diff..."


In [24]:
output_tuple = (scheduler, conditions)
fname = Path('../tmp/eq_field_survey_v0_infeasible1.p.xz').resolve()
with lzma.open(fname, 'wb') as sched_out:
    pickle.dump(output_tuple, sched_out)

fname

PosixPath('/sdf/data/rubin/user/neilsen/devel/schedview/tmp/eq_field_survey_v0_infeasible1.p.xz')

# Making pickles available

Copy them to `/sdf/group/rubin/web_data/sim-data/sched_pickles` at the USDF for them to be visible at https://s3df.slac.stanford.edu/data/rubin/sim-data/sched_pickles